# Homework #3 by Tasneem Abbasi, Myles Cook, and Keegan Smith

In [1]:
import pandas as pd
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
auctions_df = pd.read_csv('eBayAuctions.csv')
auctions_df.head()

,Category,currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,Competitive?
0,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
2,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
3,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
4,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0


In [3]:
auctions_df.dropna(inplace=True)

auctions_df['Category'] = auctions_df['Category'].astype('category')
auctions_df['currency'] = auctions_df['currency'].astype('category')
auctions_df['endDay'] = auctions_df['endDay'].astype('category')
auctions_df['Duration'] = auctions_df['Duration'].astype('category')

cat_predictors = ['Category', 'currency', 'Duration', 'endDay']
predictors = ['Category', 'currency','sellerRating', 'Duration', 'endDay', 'ClosePrice', 'OpenPrice']
outcome = 'Competitive?'

X_cat = pd.get_dummies(auctions_df[cat_predictors], dtype=int)

X = pd.concat([X_cat, auctions_df[['sellerRating', 'ClosePrice', 'OpenPrice']]], axis=1)
y = auctions_df['Competitive?']

# Problem #1a:

In [4]:
for predictor in cat_predictors:
    df = auctions_df[['Competitive?',predictor]]
    freqTable = df.pivot_table(index='Competitive?', columns=predictor,aggfunc=len)
    propTable = freqTable.apply(lambda x: x / sum(x), axis = 1)
    print(propTable)
    print()

Category      Antique/Art/Craft  Automotive     Books  Business/Industrial  \
Competitive?                                                                 
0                      0.084989    0.126932  0.029801             0.006623   
1                      0.093809    0.059099  0.025328             0.011257   

Category      Clothing/Accessories  Coins/Stamps  Collectibles  Computer  \
Competitive?                                                               
0                         0.065121      0.028698      0.111479  0.013245   
1                         0.056285      0.010319      0.129456  0.022514   

Category      Electronics  EverythingElse  Health/Beauty  Home/Garden  \
Competitive?                                                            
0                0.012141        0.014349       0.058499     0.038631   
1                0.041276        0.003752       0.010319     0.062852   

Category       Jewelry  Music/Movie/Game  Photography  Pottery/Glass  \
Competitive?     

# Problem #1b:

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

# Run logistic regression with all predictors
model = LogisticRegression(max_iter=100000)
model.fit(X_train, y_train)

# Predict on validation set
y_pred = model.predict(X_valid)

# Calculate accuracy
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy of logistic model with all predictors/features:", accuracy)

Accuracy of logistic model with all predictors/features: 0.779467680608365


# Problem #1c:

In [6]:
X_alt = X.drop(columns='ClosePrice')

In [7]:
X_train_alt, X_valid_alt, y_train_alt, y_valid_alt = train_test_split(X_alt, y, test_size=0.4, random_state=42)

model_no_price = LogisticRegression(max_iter=100000)
model_no_price.fit(X_train_alt, y_train_alt)

y_pred_no_price = model_no_price.predict(X_valid_alt)

accuracy_no_price = accuracy_score(y_valid_alt, y_pred_no_price)
print("Accuracy of logistic model excluding price:", accuracy_no_price)

Accuracy of logistic model excluding price: 0.632446134347275


Less accurate 

# Problem #1d:

In [11]:
import statsmodels.api as sm
model_eval = sm.Logit(y_train, sm.add_constant(X_train))
model_eval = model_eval.fit()
print(model_eval.summary())

         Current function value: 0.499076
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:           Competitive?   No. Observations:                 1183
Model:                          Logit   Df Residuals:                     1150
Method:                           MLE   Df Model:                           32
Date:                Mon, 26 Feb 2024   Pseudo R-squ.:                  0.2778
Time:                        22:45:22   Log-Likelihood:                -590.41
converged:                      False   LL-Null:                       -817.49
Covariance Type:            nonrobust   LLR p-value:                 4.332e-76
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                             0.4803        nan        nan        nan         nan         nan
Category

C:\Users\pff_ksmith\AppData\Local\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


The log probability of a competetive auction is increased by 0.0894 for each 1 unit (dollar) of closing price and it is statistically significant (p < 0.1)

# Problem #1e: 

In [9]:
model_regularized = LogisticRegression(penalty='l1', solver='liblinear')
model_regularized.fit(X_train, y_train)

# Predict on validation set
y_pred_regularized = model_regularized.predict(X_valid)

# Calculate accuracy
accuracy_regularized = accuracy_score(y_valid, y_pred_regularized)
print("Accuracy of regularized logistic model:", accuracy_regularized)

Accuracy of regularized logistic model: 0.7667934093789607


# Problem #1f:

We would recommend an auction with a duration of 5 days ending on either Friday or Monday given that those have the highest possitive coefficients, and that the auction use British Pounds (GBP) and is in the category "photography" although the reliance of these data is undetermined in our model. 